간단한 상황
- Weapon : 50
- Target : 100 (weight : 1~100)
- 적재량 : 3

- 상황 구현
- 에이전트 구현

kernel은 base

목적함수 대신, 새로운 결과치를 내야 한다.
1개 요격 시 보상인데, 오래 걸렸을 수록 감쇠된 보상을 얻어야 한다.
그런데 위험도에 따라 보상이 달라져야겠지.
그럼 weight 만큼의 보상을 주면 되지 않을까

np.random.seed(42)는 요격확률과 초기 위험도를 설정하게 해 줄것이다.


random.seed(42)는 요격시 실제로 요격되는지에 설정할 것이다.

In [1]:
# 필요 라이브러리 import
import collections
import copy
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [15]:
batch_size = 32
buffer_limit = 50000
gamma = 0.5
learning_rate = 0.0001


In [16]:
# WTA World 클래스
class WTAWorld():
    def __init__(self,W,T,M):
        self.W = W
        self.T = T
        self.M = M

        # Target t에 weapon w를 할당할 때의 요격확률 생성
        self.PK_wt = np.random.rand(W, T)
        # Target index 할당
        self.targets = np.array(list(range(T)))
        # Target 별 초기 위험도 할당
        self.init_weights = np.random.randint(1,100,size=T)
        # Target의 현재 위험도 설정 (0 = 파괴)
        self.weights = copy.copy(self.init_weights)


        self.M_W = [self.M] * self.W
        self.left_M_W = self.M_W
        
    def step(self, theta_wt):
        # 값이 1인 요소를 찾기
        indices = np.where(theta_wt == 1)
        
        # 좌표 형태로 출력
        coordinates = list(zip(indices[0], indices[1]))

        for coord in coordinates :
            w,t = coord
            if self.PK_wt[w][t] > 0 and self.left_M_W[w] > 0 :
                pk_probability = self.PK_wt[w][t]
                random_value = random.random()
                if random_value <  pk_probability :
                    # 요격 성공
                    self.weights[coord[1]] = 0
                else : 
                    # 요격 실패
                    pass
                self.left_M_W[w] -= 1
            else:
                pass
        
        done = self.is_done()
        r = sum(self.init_weights)-sum(self.weights)
        # print(self.init_weights)
        # print(self.weights)
        # print(sum(self.init_weights)-sum(self.weights))
        self.init_weights = copy.copy(self.weights)
        return self.PK_wt*self.weights, r,  done

    def is_done(self):
        if sum(self.left_M_W) == 0 or sum(self.weights) == 0 :
            return True
        else :
            return False
        
    def get_state(self):
        return (self.PK_wt, self.weights, self.left_M_W)
    
    def reset(self):
        # Target t에 weapon w를 할당할 때의 요격확률 생성
        self.PK_wt = np.random.rand(self.W, self.T)
        # Target index 할당
        self.targets = np.array(list(range(self.T)))
        # Target 별 초기 위험도 할당
        self.init_weights = np.random.randint(1,100,size=self.T)
        # Target의 현재 위험도 설정 (0 = 파괴)
        self.weights = copy.copy(self.init_weights)

        self.M_W = [self.M] * self.W
        self.left_M_W = self.M_W

        return self.PK_wt*self.weights

In [17]:
# Greedy Algorithm

def greedy(state):
    PK_wt = state[0]
    targets_weights = state[1]
    left_M_W = state[2]
    
    w = PK_wt.shape[0]
    t = PK_wt.shape[1]
    
    # w의 발사 여부
    m_w = [1 if m > 0 else m for m in left_M_W]
    # 할당 여부를 나타내는 theta_wt
    theta_wt = np.zeros((w, t), dtype=int)

    flattened = (PK_wt*targets_weights).flatten()
    sorted_indices = np.argsort(flattened)[::-1]
    
    i = 0
    indices_list = []
    while True :
        if i == w * t :
            break
        v = sorted_indices[i]
        row = v // t # Weapon
        col = v % t # Target

        if left_M_W[row] > 0 and m_w[row] == 1:
            
            m_w[row] -= 1
            theta_wt[row][col] = 1
            indices_list.append(v)
            print(f"weapon {row}를 target {col}에 할당, 요격확률*가중치:{flattened[v]:.2f} ")

        # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
        if sum(left_M_W) == 0:
            break

        i += 1


    return theta_wt

In [49]:
wta=WTAWorld(10,10,3)
step_i = 1
G = 0
while True : 
    print(f"{step_i}번째 스텝")
    theta_wt =greedy(wta.get_state())
    print("------발사------")
    s_prime, r, done = wta.step(theta_wt)
    print("------요격결과------")
    print(wta.get_state()[1])
    print(f"보상: {r}")
    G += r  * gamma**(step_i-1)
    if done:
        break
    step_i+=1

print(f"리턴: {G}")

1번째 스텝
weapon 3를 target 9에 할당, 요격확률*가중치:72.96 
weapon 6를 target 9에 할당, 요격확률*가중치:71.58 
weapon 1를 target 1에 할당, 요격확률*가중치:63.46 
weapon 2를 target 1에 할당, 요격확률*가중치:59.56 
weapon 0를 target 9에 할당, 요격확률*가중치:55.82 
weapon 8를 target 2에 할당, 요격확률*가중치:55.77 
weapon 5를 target 1에 할당, 요격확률*가중치:50.95 
weapon 7를 target 7에 할당, 요격확률*가중치:47.67 
weapon 4를 target 9에 할당, 요격확률*가중치:44.80 
weapon 9를 target 1에 할당, 요격확률*가중치:32.11 
------발사------
------요격결과------
[ 6  0  0 33 42 56  9  0  1  0]
보상: 277
2번째 스텝
weapon 3를 target 5에 할당, 요격확률*가중치:47.04 
weapon 7를 target 5에 할당, 요격확률*가중치:44.54 
weapon 4를 target 5에 할당, 요격확률*가중치:37.45 
weapon 1를 target 5에 할당, 요격확률*가중치:37.06 
weapon 5를 target 3에 할당, 요격확률*가중치:32.50 
weapon 2를 target 5에 할당, 요격확률*가중치:31.40 
weapon 0를 target 4에 할당, 요격확률*가중치:30.49 
weapon 6를 target 5에 할당, 요격확률*가중치:28.15 
weapon 9를 target 5에 할당, 요격확률*가중치:12.04 
weapon 8를 target 6에 할당, 요격확률*가중치:8.90 
------발사------
------요격결과------
[6 0 0 0 0 0 0 0 1 0]
보상: 140
3번째 스텝
weapon 1를 target 0에 할당, 요격확률*가중치:5.38 
weapon 

In [19]:
def main():
    env = WTAWorld(10,10,3)
    step_i = 1
    while True : 
        print(f"{step_i}번째 스텝")
        theta_wt = greedy(env.get_state())
        print("------발사------")
        env.step(theta_wt)
        print("------요격결과------")
        print(wta.get_state()[0][1])
        if env.is_done():
            break
        step_i+=1

In [20]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
        
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append(a)
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        
        # print(torch.tensor(s_lst, dtype=torch.float))
        # print(torch.stack(a_lst))
        # print(torch.tensor(r_lst))
        # print(torch.tensor(s_prime_lst, dtype=torch.float))
        # print(torch.tensor(done_mask_lst))
        return torch.tensor(s_lst, dtype=torch.float), torch.stack(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [26]:
class Qnet(nn.Module):
    def __init__(self, W, T):
        super(Qnet, self).__init__()
        self.W = W
        self.T = T
        self.fc1 = nn.Linear(self.W*self.T, self.W*self.T*20)
        self.fc2 = nn.Linear(self.W*self.T*20, self.W*self.T*20)
        self.fc3 = nn.Linear(self.W*self.T*20, self.W*self.T)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        # print(out)
        # print(out.shape)
        reshaped_out = out.reshape(self.W,self.T)
        
        one_hot = torch.zeros_like(reshaped_out)
        
        coin = random.random()
        if coin < epsilon:
            random_indices = torch.randint(0, reshaped_out.shape[1], (reshaped_out.shape[0],))
            one_hot[torch.arange(reshaped_out.shape[0]), random_indices] = 1
            return one_hot
        else :
            max_indices = torch.argmax(reshaped_out, dim=1)
            one_hot[torch.arange(reshaped_out.shape[0]), max_indices] = 1
            return one_hot

In [27]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size)
        q_out = q(s.flatten(start_dim=1))
        # print(a.flatten(start_dim=1).shape)
        # print(q_out.shape)
        q_a = a.flatten(start_dim=1) * q_out
        max_q_prime = q_target(s_prime.flatten(start_dim=1)).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        # print(target)
        # print(target.shape)
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [28]:
def main():
    env = WTAWorld(10,10,3)
    q = Qnet(env.W, env.T)
    q_target = Qnet(env.W, env.T)
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    
    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr = learning_rate)
    
    for n_epi in range(100000):
        epsilon = max(0.01, 0.08, - 0.01*(n_epi/200))
        s = env.reset()
        done = False
        
        while not done:
            a = q.sample_action(torch.from_numpy(s.flatten()).float(), epsilon)
            s_prime, r, done = env.step(a)
            # print(a.shape)
            # print(s_prime.shape)
            done_mask = 0.0 if done else 1.0
            memory.put((s, a, r, s_prime, done_mask))
            s = s_prime
            score += r
            if done:
                break
        
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        if n_epi%print_interval == 0 and n_epi !=0:
            q_target.load_state_dict(q.state_dict())
            print(f"n_episode: {n_epi}, score: {(score/print_interval):.1f}, n_buffer: {memory.size()}, eps: {(epsilon*100):.1f}%")
            score = 0.0


In [29]:
main()

n_episode: 20, score: 376.8, n_buffer: 63, eps: 8.0%
n_episode: 40, score: 340.4, n_buffer: 123, eps: 8.0%
n_episode: 60, score: 302.1, n_buffer: 182, eps: 8.0%
n_episode: 80, score: 370.1, n_buffer: 242, eps: 8.0%
n_episode: 100, score: 396.6, n_buffer: 302, eps: 8.0%
n_episode: 120, score: 376.4, n_buffer: 362, eps: 8.0%
n_episode: 140, score: 379.9, n_buffer: 421, eps: 8.0%


n_episode: 160, score: 356.4, n_buffer: 481, eps: 8.0%
n_episode: 180, score: 346.1, n_buffer: 541, eps: 8.0%
n_episode: 200, score: 361.4, n_buffer: 601, eps: 8.0%
n_episode: 220, score: 375.1, n_buffer: 661, eps: 8.0%
n_episode: 240, score: 377.6, n_buffer: 721, eps: 8.0%
n_episode: 260, score: 398.6, n_buffer: 781, eps: 8.0%
n_episode: 280, score: 366.1, n_buffer: 841, eps: 8.0%
n_episode: 300, score: 370.9, n_buffer: 901, eps: 8.0%
n_episode: 320, score: 344.5, n_buffer: 961, eps: 8.0%
n_episode: 340, score: 344.0, n_buffer: 1021, eps: 8.0%
n_episode: 360, score: 368.1, n_buffer: 1081, eps: 8.0%
n_episode: 380, score: 379.6, n_buffer: 1141, eps: 8.0%
n_episode: 400, score: 377.3, n_buffer: 1201, eps: 8.0%
n_episode: 420, score: 327.2, n_buffer: 1261, eps: 8.0%
n_episode: 440, score: 360.4, n_buffer: 1321, eps: 8.0%
n_episode: 460, score: 357.8, n_buffer: 1381, eps: 8.0%
n_episode: 480, score: 325.9, n_buffer: 1441, eps: 8.0%
n_episode: 500, score: 358.5, n_buffer: 1501, eps: 8.0%
n

/tmp/ipykernel_2584907/3681971597.py:12: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.smooth_l1_loss(q_a, target)


n_episode: 680, score: 281.1, n_buffer: 2041, eps: 8.0%
n_episode: 700, score: 137.7, n_buffer: 2101, eps: 8.0%
n_episode: 720, score: 142.2, n_buffer: 2161, eps: 8.0%
n_episode: 740, score: 118.3, n_buffer: 2221, eps: 8.0%
n_episode: 760, score: 106.0, n_buffer: 2281, eps: 8.0%
n_episode: 780, score: 95.1, n_buffer: 2341, eps: 8.0%
n_episode: 800, score: 88.2, n_buffer: 2401, eps: 8.0%
n_episode: 820, score: 105.0, n_buffer: 2461, eps: 8.0%
n_episode: 840, score: 111.0, n_buffer: 2521, eps: 8.0%
n_episode: 860, score: 103.8, n_buffer: 2581, eps: 8.0%
n_episode: 880, score: 114.8, n_buffer: 2641, eps: 8.0%
n_episode: 900, score: 88.0, n_buffer: 2701, eps: 8.0%
n_episode: 920, score: 211.6, n_buffer: 2761, eps: 8.0%
n_episode: 940, score: 146.6, n_buffer: 2821, eps: 8.0%
n_episode: 960, score: 154.4, n_buffer: 2881, eps: 8.0%
n_episode: 980, score: 94.4, n_buffer: 2941, eps: 8.0%
n_episode: 1000, score: 172.0, n_buffer: 3001, eps: 8.0%
n_episode: 1020, score: 97.5, n_buffer: 3061, eps: 

KeyboardInterrupt: 

In [ ]:
a.shape

NameError: name 'a' is not defined